In [1]:
%matplotlib qt
import napari
import aicsimageio
import matplotlib.pyplot as plt 
import numpy as np
from pathlib import Path

from utils import get_image_metadata_czi, load_image
from consts import UMBRELLA

In [2]:
rna = "90"
assert rna in {"90", "91", "scrambled"}
well = 1

experiments = list((UMBRELLA / rna / str(well)).glob("*.czi"))

In [3]:
viewer = napari.Viewer()

experiment = experiments[0]
image = load_image(str(experiment))
stack = image.get_image_data().squeeze()

average = stack.mean(axis=0)

viewer.add_image(stack, name="full stack", scale=(1, 1, 1))
viewer.add_image(average, name="average")


<Image layer 'average' at 0x16e471ee0>

In [5]:
if experiment.name == "Experiment-1363.czi" and rna == "900000":
    corner_pixels = np.array([[  0, 675, 383],[  0, 766, 484]])
elif experiment.name == 'Experiment-1354_10x.czi' and rna == "scrambled":
    corner_pixels = np.array([[  0, 528, 477], [ 0, 607, 567]])
else:
    shape = [layer for layer in viewer.layers if layer.name == "Shapes"][0]
    assert len(shape.data) == 1
    corner_pixels = shape.corner_pixels


top_left =  [corner_pixels[0, 1], corner_pixels[0, 2]]
bottom_right = [corner_pixels[1,1], corner_pixels[1,2]]
cell = stack[:, top_left[0]: bottom_right[0], top_left[1]: bottom_right[1]]

viewer.add_image(cell, name="cell")

<Image layer 'cell' at 0x15550d130>

In [6]:
plt.clf()
x = np.linspace(0, 3 * 60, cell.shape[0])
plt.plot(x, np.mean(cell, axis=(1,2)))
plt.plot(x, np.mean(cell, axis=(1,2)), '.') 

plt.xlabel("Time (s)")

plt.savefig(Path("/Users/jamesrowland/Code/dlx/figures") / f"{rna}_{well}.png")